<a href="https://colab.research.google.com/github/alannahy/DeepLabCut/blob/main/Copy_of_latest_Colab_TrainNetwork_VideoAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut Toolbox - Colab for standard (single animal) projects!

Some useful links:

- [DeepLabCut's GitHub: github.com/DeepLabCut/DeepLabCut](https://github.com/DeepLabCut/DeepLabCut)
- [DeepLabCut's Documentation: User Guide for Single Animal projects](https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html)


This notebook illustrates how to use the cloud to:
- create a training set
- train a network
- evaluate a network
- create simple quality check plots
- analyze novel videos!

### This notebook assumes you already have a project folder with labeled data!

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.


Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"

As the COLAB environments were updated to CUDA 12.X and Python 3.11, we need to install DeepLabCut and TensorFlow in a distinct way to get TensorFlow to connect to the GPU.

In [1]:
# this will take a couple of minutes to install all the dependencies!
!pip install --pre deeplabcut

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.9 MB/s eta 0:00:00
   ━━

**(Be sure to click "RESTART RUNTIME" if it is displayed above before moving on !)** You will see this button at the output of the cells above ^.

In [1]:
import deeplabcut

Loading DLC 3.0.0rc8...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


## Link your Google Drive (with your labeled data, or the demo data):

### First, place your project folder into you google drive! "i.e. move the folder named "Project-YourName-TheDate" into google drive.

In [2]:
# Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
# (We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: `/content/drive/My Drive/yourProjectFolderName`


In [24]:
# PLEASE EDIT THIS:
project_folder_name = "StairstepperTrial-AY-May5_2025"
video_type = "mp4" #, mp4, MOV, or avi, whatever you uploaded!

# No need to edit this, we are going to assume you put videos you want to analyze
# in the "videos" folder, but if this is NOT true, edit below:
videofile_path = [f"/content/drive/My Drive/{project_folder_name}/videos/"]
print(videofile_path)

# The prediction files and labeled videos will be saved in this `labeled-videos` folder
# in your project folder; if you want them elsewhere, you can edit this;
# if you want the output files in the same folder as the videos, set this to an empty string.
destfolder = f"/content/drive/My Drive/{project_folder_name}/labeled-videos"

#No need to edit this, as you set it when you passed the ProjectFolderName (above):
path_config_file = f"/content/drive/My Drive/{project_folder_name}/config.yaml"
print(path_config_file)

# This creates a path variable that links to your Google Drive project

['/content/drive/My Drive/StairstepperTrial-AY-May5_2025/videos/']
/content/drive/My Drive/StairstepperTrial-AY-May5_2025/config.yaml


## Create a training dataset:

### You must do this step inside of Colab

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models-pytorch** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pytorch_config.yaml**.

Now it is the time to start training the network!

In [25]:
# There are many more functions you can set here, including which network to use!
# Check the docstring for `create_training_dataset` for all options you can use!

deeplabcut.create_training_dataset(
  path_config_file, net_type="resnet_50", engine=deeplabcut.Engine.PYTORCH
)

INFO:root:Default augmenter albumentations not available for engine Engine.PYTORCH: using albumentations instead


/content/drive/My Drive/StairstepperTrial-AY-May5_2025/labeled-data/IMG_1364/CollectedData_AY.h5  not found (perhaps not annotated).


[(0.95,
  9,
  (array([12, 24,  3, 50,  5, 19, 27, 52,  1, 21, 22,  7, 28, 44, 18, 33,  6,
          26, 32, 13, 11, 23, 49, 53, 40, 37, 54, 36, 16, 29, 34,  8, 35, 15,
          10, 39,  0, 46, 25, 41,  9, 20, 17, 42, 47, 45,  2, 14, 43, 30, 48,
          31]),
   array([38,  4, 51])))]

## Start training:
This function trains the network for a specific shuffle of the training dataset.

In [5]:
# Let's also change the display and save_epochs just in case Colab takes away
# the GPU... If that happens, you can reload from a saved point using the
# `snapshot_path` argument to `deeplabcut.train_network`:
#   deeplabcut.train_network(..., snapshot_path="/content/.../snapshot-050.pt")

# Typically, you want to train to ~200 epochs. We set the batch size to 8 to
# utilize the GPU's capabilities.

# More info and there are more things you can set:
#   https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html#g-train-the-network

deeplabcut.train_network(
    path_config_file,
    shuffle=1,
    save_epochs=5,
    epochs=200,
    batch_size=8,
)

# This will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end.

Training with configuration:
data:
  bbox_margin: 20
  colormode: RGB
  inference:
    normalize_images: True
  train:
    affine:
      p: 0.5
      rotation: 30
      scaling: [0.5, 1.25]
      translation: 0
    crop_sampling:
      width: 448
      height: 448
      max_shift: 0.1
      method: hybrid
    gaussian_noise: 12.75
    motion_blur: True
    normalize_images: True
device: auto
metadata:
  project_path: C:\Users\yipal\Desktop\stairstepper test 1-AY-2025-05-02
  pose_config_path: C:\Users\yipal\Desktop\stairstepper test 1-AY-2025-05-02\dlc-models-pytorch\iteration-0\stairstepper test 1May2-trainset95shuffle1\train\pytorch_config.yaml
  bodyparts: ['leftleg_upperknob', 'leftleg_lowerknob', 'leftpocket_bottom', 'leftpocket_top', 'leftknee_top', 'leftshoe_knot', 'leftknee_bottom', 'thighline_bottom', 'thighline_secbottom', 'thighline_top', 'thighline_sectop', 'thighline_middle']
  unique_bodyparts: []
  individuals: ['animal']
  with_identity: None
method: bu
model:
  backbon

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

[timm/resnet50_gn.a1h_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
Data Transforms:
  Training:   Compose([
  Affine(always_apply=False, p=0.5, interpolation=1, mask_interpolation=0, cval=0, mode=0, scale={'x': (0.5, 1.25), 'y': (0.5, 1.25)}, translate_percent=None, translate_px={'x': (0, 0), 'y': (0, 0)}, rotate=(-30, 30), fit_output=False, shear={'x': (0.0, 0.0), 'y': (0.0, 0.0)}, cval_mask=0, keep_ratio=True, rotate_method='largest_box'),
  PadIfNeeded(always_apply=True, p=1.0, min_height=448, min_width=448, pad_height_divisor=None, pad_width_divisor=None, position=PositionType.CENTER, border_mode=0, value=None, mask_value=None),
  KeypointAwareCrop(always_apply=True, p=1.0, width=448, height=448, max_shift=0.1, crop_sampling='hybrid'),
  MotionBlur(always_apply=False, p=0.5, blur_limit=(3, 7), allow_shifted=True),
  GaussNoise(always_apply=False, p=0.5, var_limit=(0, 162.5625), per_channel=True, mean=0),
  Nor

Note, that **when you hit "STOP" you will get a `KeyboardInterrupt` "error"! No worries! :)**

## Start evaluating:
This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results-pytorch**

In [6]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as
# good as the labeler, so be sure your labels are good!


100%|██████████| 2/2 [00:00<00:00,  7.04it/s]


Evaluation results for DLC_Resnet50_stairstepper test 1May2shuffle1_snapshot_020-results.csv (pcutoff: 0.6):
train rmse              6.78
train rmse_pcutoff      4.21
train mAP              97.70
train mAR              98.16
test rmse               4.45
test rmse_pcutoff       4.29
test mAP              100.00
test mAR              100.00
Name: (0.95, 1, 20, -1, 0.6), dtype: float64


## There is an optional refinement step you can do outside of Colab:
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of Colab!** We recommend coming back to train and analyze videos...
- Please see the repo and protocol instructions on how to refine your data!

## Start Analyzing videos:
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides.

In [40]:
deeplabcut.analyze_videos(
    path_config_file,
    ['/content/drive/My Drive/StairstepperTrial-AY-May5_2025/videos/toanalyze/IMG_1369.MP4'],
    videotype='MP4',
    destfolder=destfolder,
)

Analyzing videos with /content/drive/My Drive/StairstepperTrial-AY-May5_2025/dlc-models-pytorch/iteration-0/stairstepper test 1May2-trainset95shuffle1/train/snapshot-best-020.pt
Starting to analyze /content/drive/My Drive/StairstepperTrial-AY-May5_2025/videos/toanalyze/IMG_1369.MP4
Video metadata: 
  Overall # of frames:    352
  Duration of video [s]:  11.73
  fps:                    30.0
  resolution:             w=1280, h=720

Running pose prediction with batch size 8


100%|██████████| 352/352 [00:48<00:00,  7.27it/s]

Saving results in /content/drive/My Drive/StairstepperTrial-AY-May5_2025/labeled-videos/IMG_1369DLC_Resnet50_stairstepper test 1May2shuffle1_snapshot_020.h5 and /content/drive/My Drive/StairstepperTrial-AY-May5_2025/labeled-videos/IMG_1369DLC_Resnet50_stairstepper test 1May2shuffle1_snapshot_020_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.



'DLC_Resnet50_stairstepper test 1May2shuffle1_snapshot_020'

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [42]:
deeplabcut.plot_trajectories(
    path_config_file,
    ['/content/drive/My Drive/StairstepperTrial-AY-May5_2025/videos/toanalyze/IMG_1369.MP4'],
    videotype='MP4',
    destfolder=destfolder,
)

Loading  /content/drive/My Drive/StairstepperTrial-AY-May5_2025/videos/toanalyze/IMG_1369.MP4 and data.
Plots created! Please check the directory "plot-poses" within the video directory


Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4.

## Create labeled video:
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

In [43]:
deeplabcut.create_labeled_video(
    path_config_file,
    ['/content/drive/My Drive/StairstepperTrial-AY-May5_2025/videos/toanalyze/IMG_1369.MP4'],
    videotype='MP4',
    destfolder=destfolder,
)

Starting to process video: /content/drive/My Drive/StairstepperTrial-AY-May5_2025/videos/toanalyze/IMG_1369.MP4
Loading /content/drive/My Drive/StairstepperTrial-AY-May5_2025/videos/toanalyze/IMG_1369.MP4 and data.
Duration of video [s]: 11.73, recorded with 30.0 fps!
Overall # of frames: 352 with cropped frame dimensions: 1280 720
Generating frames and creating video.


100%|██████████| 352/352 [00:05<00:00, 66.50it/s]


[True]